In [ ]:
# 📦 Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("../src")

from data.features import add_rolling_stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import PoissonRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
import joblib
import os

# Load cleaned data and add features
df = pd.read_csv("../data/processed/cleaned_matches.csv")
df = add_rolling_stats(df)

# Encode team names
teams = pd.concat([df['HomeTeam'], df['AwayTeam']]).unique()
team_to_id = {team: idx for idx, team in enumerate(teams)}
df['HomeTeamID'] = df['HomeTeam'].map(team_to_id)
df['AwayTeamID'] = df['AwayTeam'].map(team_to_id)

# Feature columns (no team IDs for better generalization)
feature_cols = [
    'HomeRollingGF', 'HomeRollingGA', 'HomeRollingPoints', 'HomeFormScore', 'HomeGDForm',
    'AwayRollingGF', 'AwayRollingGA', 'AwayRollingPoints', 'AwayFormScore', 'AwayGDForm'
]
X = df[feature_cols].fillna(0)  # fill missing

# Targets
y_home = df['HomeGoals']
y_away = df['AwayGoals']

# Train/Test split
X_train, X_test, y_home_train, y_home_test = train_test_split(X, y_home, test_size=0.2, random_state=42)
_, _, y_away_train, y_away_test = train_test_split(X, y_away, test_size=0.2, random_state=42)

# ⚙️ Pipelines
home_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("regressor", PoissonRegressor(alpha=0.5, max_iter=300))
])

away_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("regressor", PoissonRegressor(alpha=0.5, max_iter=300))
])

# Train
home_pipeline.fit(X_train, y_home_train)
away_pipeline.fit(X_train, y_away_train)

# Evaluate
y_home_pred = np.round(np.clip(home_pipeline.predict(X_test), 0, 6)).astype(int)
y_away_pred = np.round(np.clip(away_pipeline.predict(X_test), 0, 6)).astype(int)

print(f"Home MAE: {mean_absolute_error(y_home_test, y_home_pred):.2f}")
print(f"Away MAE: {mean_absolute_error(y_away_test, y_away_pred):.2f}")

scoreline_actual = [f"{h}-{a}" for h, a in zip(y_home_test, y_away_test)]
scoreline_pred = [f"{h}-{a}" for h, a in zip(y_home_pred, y_away_pred)]
scoreline_accuracy = np.mean(np.array(scoreline_actual) == np.array(scoreline_pred))
print(f"Exact Scoreline Accuracy: {scoreline_accuracy:.2%}")

# Save models + mapping
os.makedirs("../models", exist_ok=True)
joblib.dump(home_pipeline, "../models/home_model.pkl")
joblib.dump(away_pipeline, "../models/away_model.pkl")
joblib.dump(team_to_id, "../models/team_to_id.pkl")

print("Models and mappings saved successfully.")

Home MAE: 1.03
Away MAE: 0.86
Exact Scoreline Accuracy: 11.51%
✅ Models and mappings saved successfully.
